In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# NY data pre-processing

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)   

In [3]:
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [5]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_NY = location.latitude
longitude_NY = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_NY, longitude_NY))
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude_NY, longitude_NY], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [6]:
CLIENT_ID = 'SBDDNL3HNS3KZDQCSLRMKQFIH0A2T52ZUV0NBBTZAFU1LCQA' # your Foursquare ID
CLIENT_SECRET = 'WBWNOTRHL4OV1O3SECXFRRQLCHUM23M1VKSYZNJ0PSKR1OOJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SBDDNL3HNS3KZDQCSLRMKQFIH0A2T52ZUV0NBBTZAFU1LCQA
CLIENT_SECRET:WBWNOTRHL4OV1O3SECXFRRQLCHUM23M1VKSYZNJ0PSKR1OOJ


In [7]:
LIMIT = 100
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [9]:
print(manhattan_venues.shape)
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))
manhattan_venues.groupby('Neighborhood').count()

(3306, 7)
There are 338 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,99,99,99,99,99,99
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [10]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [11]:
manhattan_venues.shape

(3306, 7)

# Toronto data pre-processing

In [12]:
#scrape table from Wikipedia
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

#delete rows with 'Neighborhood'='Not Assigned'
df1 = df[df.Borough != "Not assigned"]

#merge row with duplicate Postcode
df2=df1.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

#set Not assigned neighborhood to be same as its Borough
df3=df2
not_assigned_index=df3.Neighborhood=='Not assigned'
df3.Neighborhood[not_assigned_index]=df3.Borough[not_assigned_index]

In [13]:
!wget -O lag_lng_coords.csv http://cocl.us/Geospatial_data
    
lag_lng_coords_df=pd.read_csv('lag_lng_coords.csv')
lag_lng_coords_df.rename(columns={"Postal Code":"Postcode"},inplace=True)

df4=pd.merge(df3,lag_lng_coords_df,on='Postcode')
df4.head()

df5=df4[df4['Borough'].str.contains("Toronto")] #Keep boroughs that contain 'Toronto'

toronto_data=df5.drop('Postcode',1)

address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude_Tor = location.latitude
longitude_Tor = location.longitude

--2020-01-01 09:33:42--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-01-01 09:33:42--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-01 09:33:44--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-01 09:33:45--  https://ibm.box.com/public/static/9afz

In [14]:
print(toronto_data.shape)
toronto_data.head()

(39, 4)


,Borough,Neighborhood,Latitude,Longitude
37,East Toronto,The Beaches,43.676357,-79.293031
41,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,East Toronto,Studio District,43.659526,-79.340923
44,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
print(manhattan_data.shape)
manhattan_data.head()

(40, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [16]:
toronto_manhattan_data=pd.concat([toronto_data,manhattan_data])
print(toronto_manhattan_data.shape)
toronto_manhattan_data.head()

(79, 4)


,Borough,Neighborhood,Latitude,Longitude
37,East Toronto,The Beaches,43.676357,-79.293031
41,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,East Toronto,Studio District,43.659526,-79.340923
44,Central Toronto,Lawrence Park,43.728020,-79.388790


In [17]:
map_toronto = folium.Map(location=[latitude_Tor, longitude_Tor], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(1675, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [20]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3306, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [21]:
toronto_manhattan_venues=pd.concat([toronto_venues,manhattan_venues])
print(toronto_manhattan_venues.shape)
toronto_manhattan_venues.head()

(4981, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [22]:
toronto_manhattan_onehot = pd.get_dummies(toronto_manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_manhattan_onehot['Neighborhood'] = toronto_manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_manhattan_onehot.columns[-1]] + list(toronto_manhattan_onehot.columns[:-1])
toronto_manhattan_onehot = toronto_manhattan_onehot[fixed_columns]
toronto_manhattan_grouped = toronto_manhattan_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_manhattan_grouped.head()

In [23]:
toronto_manhattan_grouped.shape

(78, 381)

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_manhattan_grouped['Neighborhood']

for ind in np.arange(toronto_manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Sushi Restaurant,Burger Joint,Thai Restaurant,Restaurant,American Restaurant
1,Battery Park City,Park,Coffee Shop,Hotel,Clothing Store,Gym,Wine Shop,Memorial Site,Boat or Ferry,Cupcake Shop,Food Court
2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Steakhouse,Beer Bar,Café,Bakery,Creperie
3,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Furniture / Home Store,Climbing Gym,Performing Arts Venue,Grocery Store,Gym
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Recording Studio,Auto Workshop,Skate Park,Park,Smoke Shop,Gym / Fitness Center,Burrito Place,Restaurant


In [26]:
kclusters = 10

toronto_manhattan_grouped_clustering = toronto_manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 9, 0, 0, 9, 9, 0, 9, 0, 9], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_manhattan_merged = toronto_manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_manhattan_merged = toronto_manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [28]:
toronto_manhattan_merged=toronto_manhattan_merged.dropna()
print(toronto_manhattan_merged.shape)
toronto_manhattan_merged.head() # check the last columns!

(78, 15)


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,43.676357,-79.293031,6.0,Pub,Trail,Other Great Outdoors,Health Food Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner
41,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Restaurant,Furniture / Home Store,Brewery,Bakery,Liquor Store
42,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1.0,Pizza Place,Park,Fish & Chips Shop,Italian Restaurant,Brewery,Movie Theater,Steakhouse,Burger Joint,Burrito Place,Sushi Restaurant
43,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,American Restaurant,Gastropub,Bakery,Italian Restaurant,Brewery,Bank,Bar,Latin American Restaurant
44,Central Toronto,Lawrence Park,43.728020,-79.388790,8.0,Photography Studio,Park,Swim School,Bus Line,Women's Store,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant


In [29]:
toronto_manhattan_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,43.676357,-79.293031,6.0,Pub,Trail,Other Great Outdoors,Health Food Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner
41,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Restaurant,Furniture / Home Store,Brewery,Bakery,Liquor Store
42,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1.0,Pizza Place,Park,Fish & Chips Shop,Italian Restaurant,Brewery,Movie Theater,Steakhouse,Burger Joint,Burrito Place,Sushi Restaurant
43,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,American Restaurant,Gastropub,Bakery,Italian Restaurant,Brewery,Bank,Bar,Latin American Restaurant
44,Central Toronto,Lawrence Park,43.728020,-79.388790,8.0,Photography Studio,Park,Swim School,Bus Line,Women's Store,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant
45,Central Toronto,Davisville North,43.712751,-79.390197,5.0,Park,Hotel,Breakfast Spot,Food & Drink Shop,Sandwich Place,Asian Restaurant,Gym,Clothing Store,Women's Store,Eastern European Restaurant
46,Central Toronto,North Toronto West,43.715383,-79.405678,4.0,Sporting Goods Shop,Coffee Shop,Clothing Store,Spa,Mexican Restaurant,Burger Joint,Furniture / Home Store,Café,Chinese Restaurant,Salon / Barbershop
47,Central Toronto,Davisville,43.704324,-79.388790,1.0,Dessert Shop,Sandwich Place,Café,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pizza Place,Diner,Fried Chicken Joint
48,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,3.0,Park,Tennis Court,Playground,Trail,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
49,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0.0,Pub,Coffee Shop,Sports Bar,Fried Chicken Joint,Restaurant,Sushi Restaurant,Bagel Shop,American Restaurant,Liquor Store,Supermarket


In [30]:
toronto_merged=toronto_manhattan_merged[toronto_manhattan_merged.Latitude>int(latitude_Tor)]
print(toronto_merged.shape)
toronto_merged.head()

(38, 15)


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,43.676357,-79.293031,6.0,Pub,Trail,Other Great Outdoors,Health Food Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner
41,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Restaurant,Furniture / Home Store,Brewery,Bakery,Liquor Store
42,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1.0,Pizza Place,Park,Fish & Chips Shop,Italian Restaurant,Brewery,Movie Theater,Steakhouse,Burger Joint,Burrito Place,Sushi Restaurant
43,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,American Restaurant,Gastropub,Bakery,Italian Restaurant,Brewery,Bank,Bar,Latin American Restaurant
44,Central Toronto,Lawrence Park,43.728020,-79.388790,8.0,Photography Studio,Park,Swim School,Bus Line,Women's Store,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant


In [31]:
manhattan_merged=toronto_manhattan_merged[toronto_manhattan_merged.Latitude<int(latitude_Tor)]
print(manhattan_merged.shape)
manhattan_merged.head()

(40, 15)


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0.0,Coffee Shop,Sandwich Place,Yoga Studio,Gym,Big Box Store,Steakhouse,Supplement Shop,Shopping Mall,Donut Shop,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,9.0,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Vietnamese Restaurant,Bakery,Salon / Barbershop,Optical Shop,Furniture / Home Store
2,Manhattan,Washington Heights,40.851903,-73.936900,4.0,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Spanish Restaurant,Pizza Place,Coffee Shop,Park,New American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4.0,Café,Mexican Restaurant,Bakery,Restaurant,Pizza Place,Lounge,American Restaurant,Chinese Restaurant,Pharmacy,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,9.0,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Yoga Studio,Caribbean Restaurant,Bakery,Indian Restaurant,School


In [32]:
manhattan_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0.0,Coffee Shop,Sandwich Place,Yoga Studio,Gym,Big Box Store,Steakhouse,Supplement Shop,Shopping Mall,Donut Shop,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,9.0,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Vietnamese Restaurant,Bakery,Salon / Barbershop,Optical Shop,Furniture / Home Store
2,Manhattan,Washington Heights,40.851903,-73.936900,4.0,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Spanish Restaurant,Pizza Place,Coffee Shop,Park,New American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4.0,Café,Mexican Restaurant,Bakery,Restaurant,Pizza Place,Lounge,American Restaurant,Chinese Restaurant,Pharmacy,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,9.0,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Yoga Studio,Caribbean Restaurant,Bakery,Indian Restaurant,School
5,Manhattan,Manhattanville,40.816934,-73.957385,9.0,Coffee Shop,Mexican Restaurant,Italian Restaurant,Seafood Restaurant,Park,Deli / Bodega,Supermarket,BBQ Joint,Climbing Gym,Bakery
6,Manhattan,Central Harlem,40.815976,-73.943211,9.0,Chinese Restaurant,Cosmetics Shop,Seafood Restaurant,Bar,French Restaurant,American Restaurant,African Restaurant,Café,Fried Chicken Joint,Cocktail Bar
7,Manhattan,East Harlem,40.792249,-73.944182,4.0,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,Historic Site,Park,French Restaurant,Taco Place,Beer Bar
8,Manhattan,Upper East Side,40.775639,-73.960508,9.0,Italian Restaurant,Exhibit,Coffee Shop,Bakery,Art Gallery,Gym / Fitness Center,Juice Bar,Hotel,French Restaurant,Sushi Restaurant
9,Manhattan,Yorkville,40.775930,-73.947118,9.0,Italian Restaurant,Bar,Gym,Coffee Shop,Pizza Place,Deli / Bodega,Bakery,Japanese Restaurant,Diner,Park


In [33]:
# create map NY
map_clusters = folium.Map(location=[latitude_NY, longitude_NY], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
#create map Toronto
map_clusters = folium.Map(location=[latitude_Tor, longitude_Tor], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
toronto_manhattan_merged.loc[toronto_manhattan_merged['Cluster Labels'] == 0, toronto_manhattan_merged.columns[[1] + list(range(5, toronto_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Studio District,Café,Coffee Shop,American Restaurant,Gastropub,Bakery,Italian Restaurant,Brewery,Bank,Bar,Latin American Restaurant
49,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,Sports Bar,Fried Chicken Joint,Restaurant,Sushi Restaurant,Bagel Shop,American Restaurant,Liquor Store,Supermarket
51,"Cabbagetown,St. James Town",Coffee Shop,Bakery,Pizza Place,Pharmacy,Italian Restaurant,Café,Restaurant,Pub,General Entertainment,Deli / Bodega
52,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Café,Men's Store,Mediterranean Restaurant,Yoga Studio
53,Harbourfront,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Mexican Restaurant,Café,Brewery,Event Space,Theater
54,"Ryerson,Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bakery,Middle Eastern Restaurant,Pizza Place,Bookstore,Tea Room,Theater
55,St. James Town,Café,Coffee Shop,Restaurant,Hotel,Diner,Breakfast Spot,Beer Bar,Bakery,Cocktail Bar,Clothing Store
56,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Steakhouse,Beer Bar,Café,Bakery,Creperie
57,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Juice Bar,Gym / Fitness Center,Salad Place
58,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Sushi Restaurant,Burger Joint,Thai Restaurant,Restaurant,American Restaurant


In [36]:
toronto_manhattan_merged.loc[toronto_manhattan_merged['Cluster Labels'] == 9, toronto_manhattan_merged.columns[[1] + list(range(5, toronto_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,"Harbord,University of Toronto",Café,Japanese Restaurant,Bookstore,Restaurant,Italian Restaurant,Bakery,Bar,Noodle House,Chinese Restaurant,Nightclub
67,"Chinatown,Grange Park,Kensington Market",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Chinese Restaurant,Dumpling Restaurant,Bar,Bakery,Mexican Restaurant,Grocery Store
68,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Rental Car Location,Airport Gate,Harbor / Marina,Coffee Shop,Boat or Ferry,Sculpture Garden,Airport Food Court
77,"Little Portugal,Trinity",Bar,Asian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,New American Restaurant,Café,Coffee Shop,Pizza Place,Men's Store
83,"Parkdale,Roncesvalles",Gift Shop,Bookstore,Italian Restaurant,Dog Run,Bar,Eastern European Restaurant,Dessert Shop,Restaurant,Coffee Shop,Movie Theater
87,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Recording Studio,Auto Workshop,Skate Park,Park,Smoke Shop,Gym / Fitness Center,Burrito Place,Restaurant
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Hotpot Restaurant,Vietnamese Restaurant,Bakery,Salon / Barbershop,Optical Shop,Furniture / Home Store
4,Hamilton Heights,Pizza Place,Café,Deli / Bodega,Coffee Shop,Mexican Restaurant,Yoga Studio,Caribbean Restaurant,Bakery,Indian Restaurant,School
5,Manhattanville,Coffee Shop,Mexican Restaurant,Italian Restaurant,Seafood Restaurant,Park,Deli / Bodega,Supermarket,BBQ Joint,Climbing Gym,Bakery
6,Central Harlem,Chinese Restaurant,Cosmetics Shop,Seafood Restaurant,Bar,French Restaurant,American Restaurant,African Restaurant,Café,Fried Chicken Joint,Cocktail Bar


In [37]:
toronto_manhattan_merged.loc[toronto_manhattan_merged['Cluster Labels'] == 1, toronto_manhattan_merged.columns[[1] + list(range(5, toronto_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Restaurant,Furniture / Home Store,Brewery,Bakery,Liquor Store
42,"The Beaches West,India Bazaar",Pizza Place,Park,Fish & Chips Shop,Italian Restaurant,Brewery,Movie Theater,Steakhouse,Burger Joint,Burrito Place,Sushi Restaurant
47,Davisville,Dessert Shop,Sandwich Place,Café,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pizza Place,Diner,Fried Chicken Joint
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Indian Restaurant,Café,French Restaurant,Seafood Restaurant,Chinese Restaurant,Gym,Burger Joint
35,Turtle Bay,Italian Restaurant,Sushi Restaurant,Coffee Shop,Steakhouse,Wine Bar,Park,Café,Japanese Restaurant,French Restaurant,Indian Restaurant


In [38]:
toronto_manhattan_merged.loc[toronto_manhattan_merged['Cluster Labels'] == 3, toronto_manhattan_merged.columns[[1] + list(range(5, toronto_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Moore Park,Summerhill East",Park,Tennis Court,Playground,Trail,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
50,Rosedale,Park,Trail,Playground,Electronics Store,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner


In [39]:
toronto_manhattan_merged.loc[toronto_manhattan_merged['Cluster Labels'] == 4, toronto_manhattan_merged.columns[[1] + list(range(5, toronto_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,North Toronto West,Sporting Goods Shop,Coffee Shop,Clothing Store,Spa,Mexican Restaurant,Burger Joint,Furniture / Home Store,Café,Chinese Restaurant,Salon / Barbershop
75,Christie,Café,Grocery Store,Park,Italian Restaurant,Nightclub,Candy Store,Baby Store,Diner,Athletics & Sports,Coffee Shop
76,"Dovercourt Village,Dufferin",Bakery,Supermarket,Pharmacy,Gym / Fitness Center,Park,Music Venue,Café,Recording Studio,Bar,Middle Eastern Restaurant
82,"High Park,The Junction South",Thai Restaurant,Bar,Mexican Restaurant,Café,Cajun / Creole Restaurant,Arts & Crafts Store,Park,Diner,Furniture / Home Store,Fried Chicken Joint
2,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Spanish Restaurant,Pizza Place,Coffee Shop,Park,New American Restaurant
3,Inwood,Café,Mexican Restaurant,Bakery,Restaurant,Pizza Place,Lounge,American Restaurant,Chinese Restaurant,Pharmacy,Deli / Bodega
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,Historic Site,Park,French Restaurant,Taco Place,Beer Bar
36,Tudor City,Café,Park,Mexican Restaurant,Pizza Place,Greek Restaurant,Deli / Bodega,Thai Restaurant,Diner,Dog Run,Coffee Shop
